In [1]:
import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
DATA_LOCATION = '../data/'
TRAIN_IMAGES_LOCATION = '../data/train_images/'

In [3]:
train_labels = pd.read_csv(DATA_LOCATION + 'train_onelabel.csv')
train_labels = train_labels.rename(columns={'image': 'filepath'})

In [4]:
def get_image(row):
    """
    Load image from filepath to a numpy.ndarray
    input:
        - filepath: string with relative or absolute path to image
    output:
        - img:
            numpy.ndarray containing the image
            shaped (M,N), values [0.0, 1.0]
    """
    try:
        img = TRAIN_IMAGES_LOCATION + row['filepath']
        img = mpimg.imread(img)
    except:
        img = row
        img = mpimg.imread(img)
    img = np.absolute(np.divide(img.astype(float), 255) - 1.0)
    return img

def flattened_image(row):
    img = get_image(row)
    return img.flatten().tolist()

def get_shape(row):
    img = get_image(row)
    return img.shape

In [5]:
train_labels['image'] = train_labels.apply(flattened_image, axis=1)
train_labels['shape'] = train_labels.apply(get_shape, axis=1)

In [6]:
train_labels.head()

,filepath,class,image,shape
0,132103.jpg,0,"[0.0, 0.007843137254901933, 0.0039215686274509...","(36, 51)"
1,66467.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(89, 154)"
2,9143.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(85, 103)"
3,20630.jpg,0,"[0.0, 0.0039215686274509665, 0.0, 0.0078431372...","(104, 55)"
4,33689.jpg,0,"[0.04705882352941182, 0.0039215686274509665, 0...","(52, 51)"
